# 4章　点群レジストレーション（位置合わせ）

## 4.1 最近傍点の探索（単純な方法）
- 最近傍探索・検索
    - 点群Xから任意の点p（クエリともいう）の最近傍点を見つけ出すタスク
    - xiとpの二乗距離を計算して最小のものが近傍点

In [1]:
#最近傍点探索(単純な方法)
import open3d as o3d
import numpy as np

#sin関数に従う点群の生成
X_x = np.arange(-np.pi, np.pi, 0.1)
X_y = np.sin(X_x)
X_z = np.zeros(X_x.shape)
X = np.vstack([X_x, X_y, X_z]).T

#点pを定義
p = np.array([1.0, 0.0, 0.0])

In [2]:
#XをOpen３Dの点群として用意
pcd_X = o3d.geometry.PointCloud()
pcd_X.points = o3d.utility.Vector3dVector(X)
pcd_X.paint_uniform_color([0.5, 0.5, 0.5])

#pをOpen3Dの点群として用意
pcd_p = o3d.geometry.PointCloud()
pcd_p.points = o3d.utility.Vector3dVector([p])
pcd_p.paint_uniform_color([0.0, 0.0, 1.0])

#座標軸を用意
mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()

#可視化
o3d.visualization.draw_geometries([mesh, pcd_X, pcd_p])

In [3]:
"""
点と点群の距離を計算する関数
distsの最小値とそのindexをそれぞれ取り出して返す
"""
def dist(p,X):
    #np.linalg.normを使って点pと点群Xを構成する各点までの二乗を計算し、結果をリストdistsに保存する
    dists = np.linalg.norm(p-X, axis=1)
    min_dist = min(dists)
    min_idx = np.argmin(dists)
    
    return min_dist, min_idx

In [4]:
min_dist, min_idx = dist(p, X)
np.asarray(pcd_X.colors)[min_idx] = [0.0, 1.0, 0.0]
print("distance:{:.2f}, idx:{}".format(min_dist, min_idx))
print("nearest neighbor:", X[min_idx])
o3d.visualization.draw_geometries([mesh, pcd_X, pcd_p])

distance:0.69, idx:37
nearest neighbor: [0.55840735 0.52983614 0.        ]


## 4.2 最近傍点の探索（kd-treeによる方法)
### kd-treeの構築
- k次元空間におけるデータを整理して保存するためのデータ構造
    - 木構造にすることで近傍点探索を効率化する
    - 処理対象のデータが存在する空間をを軸ごとに再帰的に区切って木構造として保持する
        　- 出来上がる気が平衡であるほど探索の処理の効率が良くなるため、分割基準は対象の軸の中央値としている

### kd-treeの探索処理
- あるクエリを準備して、kd-treeにかけて、その点とクエリの距離を半径にするとその区間内に最近傍点が存在する
    - これを繰り返すとさらに近い近傍点を見つけることもできる
 
### open3d 探索基準（パラメータ）
- search_knn_vector_3d : クエリのk近傍点を抽出する方法
- search_radius_vector_3d : 指定した半径の値以内の点を抽出する方法
- search_hybrid_vector_3d : 上記二つの基準を満たす点を抽出する方法、RKNNサーチとも呼ばれる

In [5]:
"""
search_knn_vector_3d
点群の10000番目の点をクエリとして、そこから近い順に200点を抽出している
→数字は以下のパラメータで調整できる
→dには各点の二乗距離が入っているから、ルートを取ると実際の距離になる
"""
#点群を読み込み、グレーに着色
pcd = o3d.io.read_point_cloud("../data/bun000.pcd")
pcd.paint_uniform_color([0.5, 0.5, 0.5])

#kd-treeの構築
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

query = 10000
pcd.colors[query] = [1, 0, 0]

[k, idx, d] = pcd_tree.search_knn_vector_3d(pcd.points[query], 200)
np.asarray(pcd.colors)[idx[1:], :] = [0, 0, 1]
o3d.visualization.draw_geometries([pcd])

In [6]:
"""
search_radius_vector_3d
点群の20000番目の点をクエリとして、そこから距離0.01以内の点を抽出する
"""
query = 20000
pcd.colors[query] = [1, 0, 0]
[k, idx, d] = pcd_tree.search_radius_vector_3d(pcd.points[query], 0.01)
np.asarray(pcd.colors)[idx[1:], :] = [0, 1, 0]
o3d.visualization.draw_geometries([pcd])

In [7]:
"""
search_hybrid_vector_3d
点群の5000番目の点をクエリとして、そこから距離0.01以内の点を200点抽出する
"""
query = 5000
pcd.colors[query] = [1, 0, 0]
[k, idx, d] = pcd_tree.search_hybrid_vector_3d(pcd.points[query], radius=0.01, max_nn=200)
np.asarray(pcd.colors)[idx[1:], :] = [0, 1, 1]
o3d.visualization.draw_geometries([pcd])

# 以下からは別のノートブックに移行する
- part4_ICP-point-to-point.ipynb
- part4_ICP-point-to-plane.ipynb

## 4.3 ICPアルゴリズム
Iterative Closest Points(ICP)  
- 点群同士の位置合わせ処理に広く用いられる有名なアルゴリズム  
- 位置合わせ元となるソース点群と位置合わせ先であるターゲット点群を入力とし、ソース点群をターゲット点群に位置合わせするために必要な剛体変換（回転と並進）を推定する
- 位置合わせけかに精密さを求めるケースにおいて特に効果を発揮する
- 初期値依存性のあるアルゴリズム
- ソース点群とターゲット点群がある程度近い状態でないと、いい解が得られない
  
手順
1. ソース点群とターゲット点群の対応づけ
2. 剛体変換の推定
3. 物体の姿勢のアップデート
4. 収束判定（収束しない場合は1.へ戻る）

In [8]:
#ICPアルゴリズム
import open3d as o3d
import numpy as np
import copy

pcd1 = o3d.io.read_point_cloud("../data/bun000.pcd")
pcd2 = o3d.io.read_point_cloud("../data/bun045.pcd")

#ソース点群
pcd_s = pcd1.voxel_down_sample(voxel_size=0.005)
#ターゲット点群
pcd_t = pcd2.voxel_down_sample(voxel_size=0.005)

In [9]:
pcd_s.paint_uniform_color([0.0, 1.0, 0.0])
pcd_t.paint_uniform_color([0.0, 0.0, 1.0])
o3d.visualization.draw_geometries([pcd_s, pcd_t])

ICPのアルゴリズムの実行にはo3d.pipelines.registration.registration_icpを使う　　
  
### o3d.pipelines.registration.registration_icp
引数の説明
- pcd_s : 位置合わせ元の点群
- pcd_t : 位置合わせ先の点群
- threshold : 二つの点群を対応づけする時の最大距離
- trans_init : pcd_sの初期姿勢
- obj_func : 目的関数の選択肢

In [12]:
threshold = 0.05
#今回は単位行列を初期姿勢
trans_init = np.identity(4)
obj_func = o3d.pipelines.registration.TransformationEstimationPointToPoint()
result = o3d.pipelines.registration.registration_icp(pcd_s, pcd_t, threshold, trans_init, obj_func)

In [13]:
trans_reg = result.transformation
print(trans_reg)

[[ 8.67034091e-01  4.49992090e-02 -4.96212613e-01  3.36439339e-02]
 [-3.08211182e-02  9.98849925e-01  3.67271928e-02 -4.14622696e-04]
 [ 4.97294625e-01 -1.65499006e-02  8.67423862e-01  4.06317347e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [14]:
pcd_reg = copy.deepcopy(pcd_s).transform(trans_reg)
pcd_reg.paint_uniform_color([1.0, 0.0, 0.0])
o3d.visualization.draw_geometries([pcd_reg, pcd_t])

## ICPアルゴリズムの目的関数
- Point-to-Point
    - 対応点間の二乗距離を求めている
- Point-to-Plane
    - ソースの点とターゲットの面の距離を評価する目的関数
    - 距離は点から面への最短距離
    
ただIPCアルゴリズムの動作させる時は、1行で変更できるが剛体変換を算出することができない→これは目的関数ごとに実装方法がある

In [18]:
#Point to Planeはobj_funcを変えるだけで実装できる
threshold = 0.05
#今回は単位行列を初期姿勢
trans_init = np.identity(4)
obj_func = o3d.pipelines.registration.TransformationEstimationPointToPlane()
result = o3d.pipelines.registration.registration_icp(pcd_s, pcd_t, threshold, trans_init, obj_func)
trans_reg = result.transformation
print(trans_reg)

[[ 8.32549508e-01  1.53990839e-02 -5.53736566e-01  3.45991704e-02]
 [-2.40036330e-02  9.99677504e-01 -8.28931185e-03 -1.32435428e-04]
 [ 5.53430341e-01  2.01929518e-02  8.32650649e-01  3.70841436e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


## 4.4 ICPアルゴリズムの実装（Point-to-Point）
Point-to-Pointの内部を確認する  
  
手順（再掲）
1. ソース点群とターゲット点群の対応づけ
2. 剛体変換の推定
3. 物体の姿勢のアップデート
4. 収束判定（収束しない場合は1.へ戻る）

In [20]:
#ソース点群:pcd_s, ターゲット点群:pcd_t
import open3d as o3d
import numpy as np
import numpy.linalg as LA
import copy

#データの読み込み
pcd1 = o3d.io.read_point_cloud("../data/bun000.pcd")
pcd2 = o3d.io.read_point_cloud("../data/bun045.pcd")

#ボクセル化にダウンサンプリングする
pcd_s = pcd1.voxel_down_sample(voxel_size=0.005)
pcd_t = pcd2.voxel_down_sample(voxel_size=0.005)

#初期状態を可視化
pcd_s.paint_uniform_color([0.0, 1.0, 0.0])
pcd_t.paint_uniform_color([0.0, 0.0, 1.0])
o3d.visualization.draw_geometries([pcd_t, pcd_s])

In [ ]:
#四元数から回転行列への変換とqの初期化を実装
def quanternion2rotation(q):
    rot = np.array([q[0]**2+q])